<center><h1>QUOTEMINER!! concept</h1></center>


Concept and testing notebook file for QuoteMiner. Cos jupyter variables really do come in handy cuh

In [8]:
import os
import whisperx
import subprocess
import pathlib
import torch
import json
from llama_index.core import  VectorStoreIndex, Settings, SimpleDirectoryReader
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from typing import List, Tuple
import dotenv
import unicodedata
import string
from collections import Counter
from difflib import SequenceMatcher
from concurrent.futures import ProcessPoolExecutor

import json
import pathlib
import time
from typing import Optional


In [ ]:
from pathlib import Path

output_folder = Path("./videos/")


f"{output_folder}/%(title)s.%(ext)s"

note: Should thread the youtube download script to not waste time. only network heavy


In [ ]:
import subprocess
link = "https://www.youtube.com/watch?v=hBMoPUAeLnY"
subprocess.run([
    "yt-dlp",
    "-f",
    "bv*[ext=mp4]+ba",
    "--merge-output-format",
    "mp4",
    "-o",
    "./videos/%(title)s.%(ext)s",
    link,
    "--progress"
])# -"a", "your_links.txt"

In [62]:
dotenv.load_dotenv()
#Defining constants

input_dir = pathlib.Path("./videos")
output_dir = pathlib.Path("./clips")
transcriptions_dir = pathlib.Path("./transcriptions")
hf_token = os.getenv("hugging_face_token")
#set if you have local whisperx model
model_path = None


#default pytorch variables
device = "cpu"
# batch_size = 
compute_type = "int8"
video_path = pathlib.Path("./videos/short.mp4")

In [3]:
#Check for existing directories and make if needed

os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(transcriptions_dir, exist_ok=True)

In [57]:
video_path = pathlib.Path("/Users/itzmaniss/coding/QuoteMiner/videos/Andrew Tate vs Therapist, Round #1. [2ELmm02-jCQ].mp4")

In [ ]:
# Step 1: Transcription
# def transcribe(video_path: pathlib.Path):
print("Loading MOdels\n\n")
model = whisperx.load_model("large-v2", device=device, compute_type="int8")
audio = whisperx.load_audio(str(video_path.absolute()))
print("Transcribing\n\n")
result = model.transcribe(audio, language="en", batch_size=8)
print("Segmenting\n\n")
complete_trancsript = ""
for segment in result["segments"]:
    complete_trancsript += segment["text"]    

print("Aligning\n\n")
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
refined_result = whisperx.align(result["segments"], model_a, metadata, audio, "cpu", return_char_alignments=False)
word_timestamps = []
# Iterate through each segment to extract word timestamps and confidence scores
print("Word Scoring\n\n")
for segment in refined_result["segments"]:
    for word_info in segment.get("words", []):
        if "start" in word_info and "end" in word_info:
            word_timestamps.append({
                "word": word_info["word"],
                "start": word_info["start"],  # Assuming start and end are present
                "end": word_info["end"],  # Assuming start and end are present
                "score": word_info.get("score", 0)  # Default score if not provided
            })

print("Storing Transcription Data\n\n")
transript_path = transcriptions_dir / (video_path.stem + ".json")
with open(transript_path, "w") as file:
    json.dump({"text": complete_trancsript, "word_timestampls": word_timestamps}, file)



In [4]:
from llama_index.core import Document

with open("transcriptions/Joe Rogan Experience #2223 - Elon Musk.json", "r") as file:
        data = json.load(file)

x = Document(text=data["text"])

In [ ]:

# def extract_quotes(transcriptions_dir: pathlib.Path) -> list:
Settings.llm = Ollama(model="deepseek-r1:14b")
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

# documents = SimpleDirectoryReader(input_dir=transcriptions_dir).load_data()
index = VectorStoreIndex.from_documents([x,])

with open("prompt.txt", "r") as file:
    detailed_prompt = file.read()


query_engine = index.as_query_engine()
response = query_engine.query(detailed_prompt)

y = response.response

quotes = json.loads(remove_trailing_commas(y.strip().replace("```", "").strip().replace("json", "")))

with open("motivational_lines.json", "w") as output_file:
    output_file.write(response.response)

    # return quotes



In [ ]:
y = response.response
n = remove_trailing_commas(y.strip().replace("```", "").strip().replace("json", ""))
json.loads(n)

In [30]:
y = response.response

In [ ]:
y

In [ ]:
remove_trailing_commas(y.strip().replace("```", "").strip().replace("json", ""))

In [ ]:
quotes = json.loads(remove_trailing_commas(y.strip().replace("```", "").strip().replace("json", "")))

with open("motivational_lines.txt", "w") as output_file:
    output_file.write(response.response)

In [ ]:

with open("prompt.txt", "r") as file:
    detailed_prompt = file.read()


query_engine = index.as_query_engine()
response = query_engine.query(detailed_prompt)

In [13]:
import re

def remove_trailing_commas(json_string):
    """
    Remove trailing commas from a JSON string.

    Args:
        json_string (str): The JSON string to clean.

    Returns:
        str: The JSON string without trailing commas.
    """
    # Regex to detect trailing commas before a closing brace or bracket
    trailing_comma_pattern = re.compile(r",\s*([}\]])")

    # Replace all trailing commas with the closing brace/bracket
    cleaned_json = trailing_comma_pattern.sub(r"\1", json_string)

    return cleaned_json


In [5]:
def generate_clips(video_path: pathlib.Path, quotes: List[dict]) -> None:
    with ProcessPoolExecutor() as exexutor:
        for idx, quote in enumerate(quotes):
            exexutor.submit(generate_clip, quote, idx, video_path)

    


In [4]:
def generate_clip(quote: dict, idx: int, video_path: pathlib.Path) -> None:
    start = quote["start"]
    end = quote["end"]
    if start is None or end is None:
        print(f"Skipping quote {idx + 1} due to missing timestamps.")
        return
    
    output_file = output_dir / f"{video_path.stem}_{idx + 1}.mp4"
    cmd = [
        "ffmpeg", "-i", video_path, "-ss", str(start), "-to", str(end),
        "-c", "copy", output_file
    ]
    subprocess.run(cmd)

In [ ]:
generate_clips()

In [19]:
def setup_llm_settings(retries: int = 3, timeout: int = 300) -> None:
    """
    Configure the LLM settings with retry logic and extended timeout.
    
    Args:
        retries: Number of times to retry on failure
        timeout: Timeout in seconds for the Ollama request
    """
    Settings.llm = Ollama(
        model="deepseek-r1:14b",
        timeout=600,
        request_timeout=600,  # Adding explicit request timeout
        temperature=0.7,      # Adding temperature to help with response generation
        context_window=4096,  # Explicitly setting context window
    )
    
    Settings.embed_model = HuggingFaceEmbedding(
        model_name="BAAI/bge-small-en-v1.5"
    )

def process_llm_response(response_text: str) -> dict:
    """
    Clean and parse the LLM response into JSON.
    
    Args:
        response_text: Raw response from the LLM
        
    Returns:
        Parsed JSON dictionary
    """
    cleaned_text = response_text.strip()
    cleaned_text = cleaned_text.replace("```json", "").replace("```", "")
    cleaned_text = cleaned_text.replace("json", "").strip()
    return json.loads(remove_trailing_commas(cleaned_text))

def extract_quotes(document, prompt_path: pathlib.Path, max_retries: int = 3) -> Optional[dict]:
    """
    Extract quotes from the document with retry logic.
    
    Args:
        document: Document to analyze
        prompt_path: Path to the prompt file
        max_retries: Maximum number of retry attempts
        
    Returns:
        Dictionary of extracted quotes or None if all retries fail
    """
    # Setup LLM with increased timeout
    setup_llm_settings()
    
    # Create index from document
    index = VectorStoreIndex.from_documents([document])
    
    # Read prompt
    with open(prompt_path, "r") as file:
        detailed_prompt = file.read()
    
    # Initialize query engine
    query_engine = index.as_query_engine()
    
    # Attempt query with retries
    for attempt in range(max_retries):
        try:
            response = query_engine.query(detailed_prompt)
            print(response)
            quotes = process_llm_response(response.response)
            
            # Save response to file
            with open("motivational_lines.json", "w") as output_file:
                json.dump(quotes, output_file, indent=2)
                
            return quotes
            
        except Exception as e:
            if attempt == max_retries - 1:
                print(f"Failed after {max_retries} attempts. Final error: {str(e)}")
                return None
            
            print(f"Attempt {attempt + 1} failed. Retrying after delay...")
            time.sleep(2 ** attempt)  # Exponential backoff
            
    return None


In [ ]:
quotes = extract_quotes(x, pathlib.Path("prompt.txt"))
if quotes:
    print("Successfully extracted quotes")
else:
    print("Failed to extract quotes")

In [19]:
from src.timestamp_utils import precompute_word_indices, find_best_fit_optimized


def find_timestamps(quotes, transcript_path):
    with open(transcript_path, "r") as file:
            transcript_data = json.load(file)
    word_to_indices, transcript_words = precompute_word_indices(transcript_data["text"])
    timestamped_quotes = []
    for quote in quotes:
        try:
            start, end = find_best_fit_optimized(
                quotes[quote],
                transcript_data["word_timestamps"],
                transcript_words,
                word_to_indices
            )
            timestamped_quotes.append({
                "text": quotes[quote],
                "start": start,
                "end": end
            })
        except ValueError as e:
            print(f"Could not find timestamps for quote: {quote}. Error: {e}")
    return timestamped_quotes

In [20]:
import json
import pathlib

with open("motivational_lines.json", "r") as file:
    quotes = json.load(file)
file = pathlib.Path("/Users/itzmaniss/coding/QuoteMiner/transcriptions/Joe Rogan Experience #2219 - Donald Trump.json")
timestamped_quotes = find_timestamps(quotes, file)

In [23]:
from src.video_processing import generate_clips
video_file = pathlib.Path("/Users/itzmaniss/coding/QuoteMiner/videos/Joe Rogan Experience #2219 - Donald Trump.mp4")
OUTPUT_DIR = pathlib.Path("./clips")
generate_clips(video_file, timestamped_quotes, OUTPUT_DIR)

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1_4 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --e

In [21]:
timestamped_quotes

[{'text': 'Nobody gives up prime time for being president. They told me I was crazy. The Apprentice was still a hit, NBC wanted me to stay, but I walked away because I knew I had to do this.',
  'start': 135.379,
  'end': 147.889},
 {'text': 'The moment I started winning, everything changed. The same people who once praised me, hugged me, and called me a friend turned against me overnight. It wasn’t just politics—it was the machine at work.',
  'start': 248.801,
  'end': 259.863},
 {'text': 'When I got shot, it wasn’t surreal. When I became president, that was surreal. Lying on the ground, I knew exactly what was happening. But standing in the White House for the first time, looking around, realizing I was president—that was something else entirely.',
  'start': 306.329,
  'end': 320.22},
 {'text': 'You don’t think about the weight of the office until you’re standing in the Lincoln Bedroom, looking at the Gettysburg Address, knowing you now hold the responsibility of history in your ha